In [1]:
!apt-get update && apt-get install -y ffmpeg

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]        
Get:4 https://packages.cloud.google.com/apt gcsfuse-focal InRelease [1225 B]   
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Get:6 https://packages.cloud.google.com/apt cloud-sdk InRelease [1616 B]
Get:7 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1252 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [3817 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1539 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [3844 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [3969 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [4310 kB]
Get:13 https://packages.cloud.google.com/apt g

In [3]:
!ffmpeg -i /kaggle/input/sample-bengaluru-mobility/18th_Crs_BsStp_JN_FIX_1_000.mp4 -vf fps=1 /kaggle/working/frame_%04d.png

['ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers',
 '  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)',
 '  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-lib

In [4]:
!pip install torch torchvision

In [7]:
import os
import glob
import torch
from torchvision import transforms
from PIL import Image

In [5]:
frames_dir = '/kaggle/working/'

frame_files = sorted([os.path.join(frames_dir, file) for file in os.listdir(frames_dir) if file.endswith('.png')])

transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

# Load and convert all frames to tensors
frames_tensor = torch.stack([transform(Image.open(file)) for file in frame_files])

print(frames_tensor.shape)  

torch.save(frames_tensor, '/kaggle/working/frames_tensor.pt')

torch.Size([900, 3, 224, 224])


In [8]:
png_files = glob.glob('/kaggle/working/*.png')

for file in png_files:
    os.remove(file)

print(f"Deleted {len(png_files)} .png files from /kaggle/working directory.")

Deleted 900 .png files from /kaggle/working directory.


In [9]:
!ffmpeg -i /kaggle/input/sample-bengaluru-mobility/18th_Crs_BsStp_JN_FIX_1_000.mp4 -vf "fps=0.5,scale=640:360" /kaggle/working/frame_%04d.png

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [10]:
frames_dir = '/kaggle/working/'

frame_files = sorted([os.path.join(frames_dir, file) for file in os.listdir(frames_dir) if file.endswith('.png')])

transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

#float16
frames_tensor = torch.stack([transform(Image.open(file)).half() for file in frame_files])

print(frames_tensor.shape)  

torch.save(frames_tensor, '/kaggle/working/frames_tensor_float16.pt')

torch.Size([450, 3, 224, 224])


In [11]:
png_files = glob.glob('/kaggle/working/*.png')

for file in png_files:
    os.remove(file)

print(f"Deleted {len(png_files)} .png files from /kaggle/working directory.")

Deleted 450 .png files from /kaggle/working directory.


In [13]:
import h5py
import numpy as np

In [12]:
frames_np = frames_tensor.numpy()

with h5py.File('/kaggle/working/frames_tensor.h5', 'w') as f:
    f.create_dataset('frames', data=frames_np, compression="gzip", compression_opts=9)